In [1]:
import os

In [2]:
%pwd

'e:\\Retail-Analysis-with-Walmart\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Retail-Analysis-with-Walmart'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str


In [7]:
from real_walmart.constants import *
from real_walmart.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            model_name=config.model_name
        )

        return model_trainer_config


In [9]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestRegressor
from real_walmart.logging import logger


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        try:
            logger.info("Starting model training")

            # Load training data
            train_df = pd.read_csv(self.config.train_data_path)

            X_train = train_df.drop(columns=["Weekly_Sales"])
            y_train = train_df["Weekly_Sales"]

            # Train model
            model = RandomForestRegressor(
                n_estimators=200,
                random_state=42
            )
            model.fit(X_train, y_train)

            # Save model
            os.makedirs(self.config.root_dir, exist_ok=True)
            model_path = os.path.join(self.config.root_dir, self.config.model_name)
            joblib.dump(model, model_path)

            logger.info(f"Model training completed. Model saved at {model_path}")

        except Exception as e:
            raise e


In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e


[2025-12-20 09:19:11,520: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-12-20 09:19:11,528: INFO: common: YAML file loaded successfully: params.yaml]
[2025-12-20 09:19:11,533: INFO: common: Created directory at: artifacts]
[2025-12-20 09:19:11,535: INFO: common: Created directory at: artifacts/model_trainer]
[2025-12-20 09:19:11,538: INFO: 3974917609: Starting model training]
[2025-12-20 09:19:17,135: INFO: 3974917609: Model training completed. Model saved at artifacts/model_trainer\model.pkl]
